In [6]:
import os
import pandas as pd
import numpy as np
import datetime
import capstone_twitter_search
from capstone_twitter_search import twittsearch


from nltk.tokenize import word_tokenize          # to divide strings into tokens

import tensorflow as tf            

import numpy as np

import text_proccessing
from text_proccessing import clean_text
from text_proccessing import remove_stopwords
from text_proccessing import lemmatize_text
from text_proccessing import concatenate_text
from text_proccessing import makeglove
from text_proccessing import make_embedding_matrix
from text_proccessing import processme

import modhelp
from modhelp import train_val_split
from modhelp import geo_df
from modhelp import suggest_nn2

In [7]:
code_dir = os.getcwd()
parent_dir = os.path.dirname(code_dir)
data_dir = os.path.join(parent_dir,"Data")
directory = os.path.join(data_dir,'splits')
dsa= os.path.dirname(parent_dir)
tweet_dir = os.path.join(parent_dir,"TweetMap")


In [8]:
train_data = pd.read_csv(os.path.join(directory, 'InformativenessTrain_Tokenized.csv'),engine='python')
train_data['text']= train_data['text'].astype(str)

In [9]:
start =  datetime.datetime.now() 
print('splitthis started at: {}'.format(start))
train_samples, val_samples, train_labels, val_labels = train_val_split(train_data, 0.25)
end =  datetime.datetime.now() 
print ('Text done {}'.format(end-start))
start =  datetime.datetime.now() 
print('Loading model,embedding and glove commenced at  {}'.format(start))

model = tf.keras.models.load_model(os.path.join(dsa,'model1'))
print("loading vecotor")
path_to_glove_file = os.path.join(dsa,'WordVector','glove.twitter.27B.200d.txt')

print("indexing")
embeddings_index=makeglove(path_to_glove_file)
print("matrix and vectorize")
embedding_matrix, vectorizer = make_embedding_matrix(train_samples, val_samples, embeddings_index)
end =  datetime.datetime.now() 

print("embedded in {}".format(end-start))

splitthis started at: 2021-07-16 19:39:20.432934
Total size of the dataset: 117339.
Training dataset: 88005.
Validation dataset: 29334.
Text done 0:00:00.813266
Loading model,embedding and glove commenced at  2021-07-16 19:39:21.247367
loading vecotor
indexing
matrix and vectorize
Converted 32255 words (22745 misses).
embedded in 0:03:03.237175


In [ ]:
scrapestart =  datetime.datetime.now() 
print("Scraping commenced at {}".format(scrapestart))
# =============================================================================
text_query = '("forest fire") OR wildfire OR bushfire OR \
(extreme heat) OR (record heat) OR heatwave OR ("heat wave") OR typhoon OR cyclone OR hurricane OR \
tornado OR ("storm surge") OR  blizzard OR snow OR ("ice storm") OR sleet OR thunderstorm OR \
hail OR flood OR flooding OR freeze OR frost OR (extreme cold) OR landslide OR tsunami OR ("tidal wave") OR \
earthquake OR eruption OR volcano OR lava OR lahar OR avalanche OR mudslide OR sinkhole'


# qlist = [firemetq_geologicalotherq]
# qname= ['fire & meteorlogical & geological & other']
since_date = '2021-07-11'
until_date = '2021-07-12'
tweetcount = 150000
twts = twittsearch(text_query, since_date,until_date,tweetcount)
#  twittsearch(text_query,since_date,until_date,tweetcount).to_csv(os.path.join(tweet_dir,'tweets_df_{0}_{1}.csv'.
#                               format(since_date.replace('-',''),until_date.replace('-',''))),
#                  index =False)

scrapend =  datetime.datetime.now()
print("Scraping ended at {}".format(scrapend))
print("Scraping time {}".format(scrapend-scrapestart))

twts = processme(twts)
scrapallend =  datetime.datetime.now()
print("{}: Scraping time {}".format(scrapallend, scrapallend-scrapestart))

Scraping commenced at 2021-07-16 19:42:24.927057
("forest fire") OR wildfire OR bushfire OR (extreme heat) OR (record heat) OR heatwave OR ("heat wave") OR typhoon OR cyclone OR hurricane OR tornado OR ("storm surge") OR  blizzard OR snow OR ("ice storm") OR sleet OR thunderstorm OR hail OR flood OR flooding OR freeze OR frost OR (extreme cold) OR landslide OR tsunami OR ("tidal wave") OR earthquake OR eruption OR volcano OR lava OR lahar OR avalanche OR mudslide OR sinkhole since:2021-07-11 until:2021-07-12 filter:has_engagement


In [ ]:


predstart = datetime.datetime.now() 
print("{}: predict".format(predstart))
predictions = suggest_nn2(twts, model,vectorizer)

submission_data = {"ID": twts['TweetId'].tolist(),"tweet": twts['Text'].tolist(), "target": predictions}

submission_df = pd.DataFrame(submission_data)
result = twts.join(submission_df.target)
predend = datetime.datetime.now() 
print("predicted {}".format(predend-predstart))
# 
result_inf = result[result['target']==0]
print("working over {} informative tweets".format(len(result_inf)))



In [ ]:


print("Run geoprocessing over tweets")

geostart =  datetime.datetime.now() 
print("{}: geostart".format(geostart))
df_js = geo_df(result_inf)
geoend =  datetime.datetime.now()
print("{}: geoend {}".format(geoend,geoend-geostart))

df_js.to_csv(os.path.join(tweet_dir,'result_{0}_{1}.csv'.
                              format(since_date.replace('-',''),until_date.replace('-',''))))
print("fín")